# **CHAP8.텍스트 분석** 

## **5. 감성분석**

### 1) 감성분석 소개

* 문서의 주관적인 감성/의견/감정/기분 등을 파악하기 위한 방법으로 소셜 미디어, 여론조사, 온라인 리뷰, 피드백 등 다양한 분야에서 활용됨

* 문서 내 텍스트가 나타내는 여러가지 주관적인 단어와 문맥을 기반으로 감성 수치를 계산하는 방법을 이용

* 감성지수 - 긍정감성지수/부정감성지수

* 지도학습 방법 : 학습 데이터와 타깃 레이블 값을 기반으로 감성 분석 학습을 수행한 뒤 이를 기반으로 다른 데이터의 감성 분석을 예측하는 방법으로 일반적인 텍스트 기반의 분류와 거의 동일

* 비지도 학습 방법 : 감성 어휘 사전 이용(Lexicon)

### 2) 지도학습 기반 감성 분석 실습 - IMDB 영화평

* IMDB의 영화 사이트의 영화평 이용
* 텍스트 기반의 이진 분류와 비슷

In [99]:
import pandas as pd

review_df=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ESAA/OB/DataSet/labeledTrainData.tsv',header=0, sep="\t", quoting=3)
review_df.head(3)

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."


In [100]:
print(review_df['review'][0])

"With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally sta

* <br \>태그 삭제
* 숫자, 특수문자 제거 (정규표현식 이용)


In [101]:
import re
review_df['review']=review_df['review'].str.replace('<br />',' ')

review_df['review']=review_df['review'].apply(lambda x:re.sub("[^a-zA-Z]"," ",x))

* sentiment 칼럼 별도 추출해 결정 값 데이터 세트 만들기

* 원본 데이터세트에서 id와 sentiment 칼럼을 삭제해 피처 데이터 세트 생성

In [102]:
from sklearn.model_selection import train_test_split

class_df=review_df['sentiment']
feature_df=review_df.drop(['id','sentiment'], axis=1, inplace=False)

X_train, X_test, y_train, y_test = train_test_split(feature_df, class_df, test_size=0.3, random_state=156)

X_train.shape, X_test.shape

((17500, 1), (7500, 1))

* train : 17500개, test : 7500개

* ML 분류 알고리즘을 적용해 예측 성능 측정 

In [103]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score

pipeline = Pipeline([
    ('cnt_vect', CountVectorizer(stop_words = 'english', ngram_range=(1,2))),
    ('lr_clf',LogisticRegression(C = 10))])

pipeline.fit(X_train['review'], y_train)
pred = pipeline.predict(X_test['review'])

pred_probs = pipeline.predict_proba(X_test['review'])[:,1]
print(accuracy_score(y_test, pred), roc_auc_score(y_test, pred_probs))   

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


0.886 0.9502705298201685


* TF-IDF 벡터화를 적용해 예측성능 측정

In [104]:
pipeline = Pipeline([
    ('tfidf_vect',TfidfVectorizer(stop_words = 'english', ngram_range=(1,2)) ),
    ('lr_clf',LogisticRegression(C= 10))])

pipeline.fit(X_train['review'], y_train)
pred = pipeline.predict(X_test['review'])

pred_probas = pipeline.predict_proba(X_test['review'])[:,1]

print(accuracy_score(y_test, pred), roc_auc_score(y_test,pred_probas))

0.8936 0.959799823582973


### 3) 비지도학습 기반 감성 분석 소개

* 감성 어휘 사전 Lexicon 기반
* WordNet : 시맨틱 분석을 제공하는 어휘 사전
* 시맨틱 : 문맥상 의미
* 감성 사전 : SentiWordNet, VADER, Pattern

### 4) SentiWordNet을 이용한 감성 분석

In [105]:
import nltk
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package basque_grammars is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Pac

True

In [106]:
from nltk.corpus import wordnet as wn

term='present'

# 'present'라는 단어로 wordnet의 synsets 생성
synsets=wn.synsets(term)
print('synsets() 반환 type :', type(synsets))
print('synsets() 반환 값 개수 : ', len(synsets))
print('synsets() 반환 값 : ', synsets)

synsets() 반환 type : <class 'list'>
synsets() 반환 값 개수 :  18
synsets() 반환 값 :  [Synset('present.n.01'), Synset('present.n.02'), Synset('present.n.03'), Synset('show.v.01'), Synset('present.v.02'), Synset('stage.v.01'), Synset('present.v.04'), Synset('present.v.05'), Synset('award.v.01'), Synset('give.v.08'), Synset('deliver.v.01'), Synset('introduce.v.01'), Synset('portray.v.04'), Synset('confront.v.03'), Synset('present.v.12'), Synset('salute.v.06'), Synset('present.a.01'), Synset('present.a.02')]


* synsets가 가지는 속성 확인

In [107]:
for synset in synsets:
    print('#####Synset name : ', synset.name(), '####') 
    print('POS : ', synset.lexname())
    print('Definition : ', synset.definition()) 
    print('Lemmas:', synset.lemma_names()) 

#####Synset name :  present.n.01 ####
POS :  noun.time
Definition :  the period of time that is happening now; any continuous stretch of time including the moment of speech
Lemmas: ['present', 'nowadays']
#####Synset name :  present.n.02 ####
POS :  noun.possession
Definition :  something presented as a gift
Lemmas: ['present']
#####Synset name :  present.n.03 ####
POS :  noun.communication
Definition :  a verb tense that expresses actions or states at the time of speaking
Lemmas: ['present', 'present_tense']
#####Synset name :  show.v.01 ####
POS :  verb.perception
Definition :  give an exhibition of to an interested audience
Lemmas: ['show', 'demo', 'exhibit', 'present', 'demonstrate']
#####Synset name :  present.v.02 ####
POS :  verb.communication
Definition :  bring forward and present to the mind
Lemmas: ['present', 'represent', 'lay_out']
#####Synset name :  stage.v.01 ####
POS :  verb.creation
Definition :  perform (a play), especially on a stage
Lemmas: ['stage', 'present', 're

* 어휘 관계 유사도 확인

In [108]:
tree = wn.synset('tree.n.01')
lion = wn.synset('lion.n.01')
tiger = wn.synset('tiger.n.02')
cat = wn.synset('cat.n.01')
dog = wn.synset('dog.n.01')

entities = [tree, lion, tiger, cat, dog]

similarities = []

entity_names = [entity.name().split('.')[0] for entity in entities]

for entity in entities:
    similarity = [round(entity.path_similarity(compared_entity) ,2) for compared_entity in entities]
    
    similarities.append(similarity)
    
    
similarity_df = pd.DataFrame(similarities, columns=entity_names, index = entity_names)
similarity_df

,tree,lion,tiger,cat,dog
tree,1.00,0.07,0.07,0.08,0.12
lion,0.07,1.00,0.33,0.25,0.17
tiger,0.07,0.33,1.00,0.25,0.17
cat,0.08,0.25,0.25,1.00,0.20
dog,0.12,0.17,0.17,0.20,1.00


In [109]:
import nltk
from nltk.corpus import sentiwordnet as swn

senti_synsets = list(swn.senti_synsets('slow'))
print('senti_synsets() 반환 type :', type(senti_synsets))
print('senti_synsets() 반환 값 개수 : ', len(senti_synsets))
print('senti_synsets() 반환 값 : ', senti_synsets)

senti_synsets() 반환 type : <class 'list'>
senti_synsets() 반환 값 개수 :  11
senti_synsets() 반환 값 :  [SentiSynset('decelerate.v.01'), SentiSynset('slow.v.02'), SentiSynset('slow.v.03'), SentiSynset('slow.a.01'), SentiSynset('slow.a.02'), SentiSynset('dense.s.04'), SentiSynset('slow.a.04'), SentiSynset('boring.s.01'), SentiSynset('dull.s.08'), SentiSynset('slowly.r.01'), SentiSynset('behind.r.03')]


* 감성 지수와 객관성 지수

In [110]:
import nltk
from nltk.corpus import sentiwordnet as swn

father=swn.senti_synset('father.n.01')
print('father 긍정감성지수 : ',father.pos_score())
print('father 부정감성지수 :', father.neg_score())
print('father 객관성 지수 :', father.obj_score())
print('\n')
fabulous=swn.senti_synset('fabulous.a.01')
print('fabulous 긍정감성 지수 : ', fabulous.pos_score())
print('fabulous 부정감성 지수 : ', fabulous.neg_score())

father 긍정감성지수 :  0.0
father 부정감성지수 : 0.0
father 객관성 지수 : 1.0


fabulous 긍정감성 지수 :  0.875
fabulous 부정감성 지수 :  0.125


#### (1) SentiWordNet을 이용한 영화 감상평 분석

* 분석 과정
  * 문서를 문장 단위로 분해
  * 다시 문장을 단어 단위로 토큰화하고 품사 태깅
  * 품사 태깅괸 단어 기반으로 synset 객체와 senti_synset 객체를 생성
  * Senti_synset에서 긍정 감성/ 부정 감성 지수를 구하고 이를 모두 합산해 특정 임계치 값 이상일 때 긍정 감성으로, 그렇지 않을 때는 부정 감성으로 결정

In [111]:
from nltk.corpus import wordnet as wordnet

# 간단산 NTLK PennTreevank Tag 를 기반으로 WordNet 기반의 품사 Tag로 변환
def penn_to_wn(tag):
  if tag.startswith('J'):
    return wn.ADJ
  elif tag.startswith('N'):
    return wn.NOUN
  elif tag.startswith('R'):
    return wn.ADV
  elif tag.startswith('V'):
    return wn.VERB

In [118]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import sentiwordnet as swn
from nltk import sent_tokenize, word_tokenize, pos_tag

def swn_polarity(text):
    # 감성 지수 초기화 
    sentiment = 0.0
    tokens_count = 0
    
    lemmatizer = WordNetLemmatizer()
    raw_sentences = sent_tokenize(text)

    # 분해된 문장별로 단어 토큰 -> 품사 태깅 후에 SentiSynset 생성 -> 감성 지수 합산 
    for raw_sentence in raw_sentences:

        # NTLK 기반의 품사 태깅 문장 추출  
        tagged_sentence = pos_tag(word_tokenize(raw_sentence))

        for word , tag in tagged_sentence:
            # WordNet 기반 품사 태깅과 어근 추출
            wn_tag = penn_to_wn(tag)
            if wn_tag not in (wn.NOUN , wn.ADJ, wn.ADV):
                continue  

            lemma = lemmatizer.lemmatize(word, pos=wn_tag) 
            if not lemma:
                continue

            # 어근을 추출한 단어와 WordNet 기반 품사 태깅을 입력해 Synset 객체를 생성. 
            synsets = wn.synsets(lemma , pos=wn_tag)
            if not synsets:
                continue
                
            # sentiwordnet의 감성 단어 분석으로 감성 synset 추출
            # 모든 단어에 대해 긍정 감성 지수는 +로 부정 감성 지수는 -로 합산해 감성 지수 계산. 
            synset = synsets[0]
            swn_synset = swn.senti_synset(synset.name())
            sentiment += (swn_synset.pos_score() - swn_synset.neg_score())           
            tokens_count += 1
    
    if not tokens_count:
        return 0
    
    # 총 score가 0 이상일 경우 긍정(Positive) 1, 그렇지 않을 경우 부정(Negative) 0 반환
    if sentiment >= 0 :
        return 1
    
    return 0

In [119]:
review_df['preds'] = review_df['review'].apply( lambda x : swn_polarity(x) )
y_target = review_df['sentiment'].values
preds = review_df['preds'].values

In [120]:
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score
from sklearn.metrics import recall_score, f1_score, roc_auc_score
import numpy as np

print(confusion_matrix(y_target,preds))
print('정확도:',np.round(accuracy_score(y_target, preds),4))
print('정밀도:', np.round(precision_score(y_target,preds),4))
print('재현율:',np.round(recall_score(y_target,preds),4))

[[7668 4832]
 [3636 8864]]
정확도: 0.6613
정밀도: 0.6472
재현율: 0.7091


### 5) VADER를 이용한 감성분석

In [121]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

senti_analyzer=SentimentIntensityAnalyzer()
senti_scores=senti_analyzer.polarity_scores(review_df['review'][0])
print(senti_scores)


{'neg': 0.13, 'neu': 0.743, 'pos': 0.127, 'compound': -0.7943}


* SentimentIntensityAnalyzer 객체를 생성한 뒤 문서별로 polarity_scores()메서드를 호출해 감성 점수를 구한 뒤, 해당 문서의 감성 점수가 특정 임계값 이상이면 긍정, 그렇지 않으면 부정으로 판단

In [122]:
def vader_polarity(review, threshold=0.1):
  analyzer=SentimentIntensityAnalyzer()
  scores=analyzer.polarity_scores(review)

  agg_score=scores['compound']
  final_sentiment=1 if agg_score>=threshold else 0
  return final_sentiment

review_df['vader_preds']=review_df['review'].apply(lambda x : vader_polarity(x,0.1))
y_target=review_df['sentiment'].values
vader_preds=review_df['vader_preds'].values

print(confusion_matrix(y_target, vader_preds))
print('정확도:', np.round(accuracy_score(y_target, vader_preds),4))
print('정밀도:', np.round(precision_score(y_target,preds),4))
print('재현율:',np.round(recall_score(y_target,preds),4))


[[ 6747  5753]
 [ 1858 10642]]
정확도: 0.6956
정밀도: 0.6472
재현율: 0.7091


* 정확도가 SentiWordNet보다 향상됐고, 재현율이 매우 크게 향상됨